In [3]:
import os
import time
import threading

#### Current Thread

In [3]:
print(f'Program enter, Process id: {os.getpid()}')
print(f'Current Thread name: {threading.current_thread().name}')
print('Sleeping...')
time.sleep(4) # current thread will sleep
print('Wake up...')

threading.current_thread().name = 'MyThread'

print(f'Current Thread name: {threading.current_thread().name}')

print('Program exit')

Program enter, Process id: 1907
Current Thread name: MainThread
Sleeping...
Wake up...
Current Thread name: MyThread
Program exit


#### Create a new Thread
- Create an instance of Thread class and associate a target function which will be executed by the Thread when running 
- Write a sub-class of Thread and instantiate the sub-class

In [10]:
def run():
    for i in range(1,6):
        print(f'{i} --> Thread name: {threading.current_thread().name} - Process id: {os.getpid()}')
        time.sleep(1)

In [22]:
print(f'Main Thread enter, Process id: {os.getpid()}')

t1 = threading.Thread(target=run, name='MyThread-01')
t1.start()

print('t1 is alive:', t1.is_alive())

t1.join() # Current thread (main) goes to wait state, gets notified when t1 is dead

print('t1 is alive:', t1.is_alive())

print('Main Thread exit')

Main Thread enter, Process id: 1930
1 --> Thread name: MyThread-01 - Process id: 1930
t1 is alive: True
2 --> Thread name: MyThread-01 - Process id: 1930
3 --> Thread name: MyThread-01 - Process id: 1930
4 --> Thread name: MyThread-01 - Process id: 1930
5 --> Thread name: MyThread-01 - Process id: 1930
t1 is alive: False
Main Thread exit


In [24]:
print(f'Main Thread enter, Process id: {os.getpid()}')

t1 = threading.Thread(target=run, name='First')
t2 = threading.Thread(target=run, name='Second')

t1.start()
t2.start()

t1.join()
t2.join()

print('Main Thread exit')

Main Thread enter, Process id: 1930
1 --> Thread name: First - Process id: 1930
1 --> Thread name: Second - Process id: 1930
2 --> Thread name: First - Process id: 1930
2 --> Thread name: Second - Process id: 1930
3 --> Thread name: First - Process id: 1930
3 --> Thread name: Second - Process id: 1930
4 --> Thread name: First - Process id: 1930
4 --> Thread name: Second - Process id: 1930
5 --> Thread name: First - Process id: 1930
5 --> Thread name: Second - Process id: 1930
Main Thread exit


#### Create a Thread whose target function accepts numbers, add them and display the sum along with Thread name 

In [33]:
def add(*args):
    print(f'Thread name: {threading.current_thread().name}, Sum: {sum(args)}')

In [36]:
t1 = threading.Thread(target=add, name='First', args=(1,2,3,4,5))
t2 = threading.Thread(target=add, name='Second', args=(10,20,30))

t1.start()
t2.start()

Thread name: First, Sum: 15
Thread name: Second, Sum: 60


#### Creating a Thread by Inheritance
1. Write a sub-class of Thread 
2. Target method by default would be instance method named 'run' and we override the method
3. t.run is the way to specify the target in case we don't override run method and intend to have other method as target
4. Arguments can be set explictly through args attribute (t.args) and are accessed in the Thread class as self.args

In [48]:
class MyThread(threading.Thread):
    def run(self):
        for i in range(1,6):
            print(f'{i} --> Thread name: {threading.current_thread().name} - Process id: {os.getpid()}')
            time.sleep(1)
#--------------------------------------
t1 = MyThread()
t1.name = 'First'
t1.start()            

1 --> Thread name: First - Process id: 1930
2 --> Thread name: First - Process id: 1930
3 --> Thread name: First - Process id: 1930
4 --> Thread name: First - Process id: 1930
5 --> Thread name: First - Process id: 1930


In [49]:
class MyThread(threading.Thread):
    def do_something(self):
        for i in range(1,6):
            print(f'{i} --> Thread name: {threading.current_thread().name} - Process id: {os.getpid()}')
            time.sleep(1)
#--------------------------------------
t1 = MyThread()
t1.name = 'First'
t1.run = t1.do_something # set the target for our Thread
t1.start()

1 --> Thread name: First - Process id: 1930
2 --> Thread name: First - Process id: 1930
3 --> Thread name: First - Process id: 1930
4 --> Thread name: First - Process id: 1930
5 --> Thread name: First - Process id: 1930


#### Write a sub class of Thread whose target sums the arguments and display the sum along with the Thread name

In [52]:
class AddThread(threading.Thread):
    def run(self):
        print(f'Thread: {threading.current_thread().name}, Sum: {sum(self.args)}')

#----------------------------------

t1 = AddThread()
t1.name = 'First'
t1.args = (1,2,3)

t2 = AddThread()
t2.name = 'Second'
t2.args = (11, 22, 33)

t1.start()
t2.start()

Thread: First, Sum: 6
Thread: Second, Sum: 66


#### Thread assignment
- Write a class named Resource (not a sub-class of Thread)
- It will have an instance field named 'data', initialized to 0
- It will have an instance method named 'do_something'
- The method will increment data by 1, display current thread name along with current value of data
- The method will be target method for a Thread

In [81]:
class Resource:
    def __init__(self):
        self.data = 0
        
    def do_something(self):
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} - {self.data}')

In [79]:
class Resource:
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock()
        
    def do_something(self):
        self.lock.acquire()
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} - {self.data}')
        self.lock.release()        

In [83]:
class Resource:
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock()
        
    def do_something(self):
        with self.lock:
            self.data += 1
            for i in range(1000000):
                pass
            print(f'{threading.current_thread().name} - {self.data}')

In [84]:
r = Resource()

t1 = threading.Thread(target=r.do_something, name='First')
t2 = threading.Thread(target=r.do_something, name='Second')
t3 = threading.Thread(target=r.do_something, name='Third')

t1.start()
t2.start()
t3.start()

First - 1
Second - 2
Third - 3


#### MultiProcessing in Python allows us to run tasks in parallel (one task in each process)
- MultiThreading - Concurrent processing (uses only one CPU which is shared by all the Threads) - GIL
- MultiProcessing - Parallel processing (uses as many CPUs as available)

In [85]:
import os
import multiprocessing

print(f'Available processors: {os.cpu_count()}')

Available processors: 8


In [87]:
def worker1():
    print(f'In Worker-1 target, Process id: {os.getpid()}')
    
def worker2():
    print(f'In Worker-2 target, Process id: {os.getpid()}')

#---------------------------------------------------------------

print(f'Main Process , Process id: {os.getpid()}')

p1 = multiprocessing.Process(target=worker1)
p2 = multiprocessing.Process(target=worker2)

p1.start()
p2.start()

p1.join()
p2.join()

print('End of Main Process')

Main Process , Process id: 1930
In Worker-2 target, Process id: 2381
In Worker-1 target, Process id: 2379
End of Main Process


#### Create two Process objects
- One will square a given number (as parameter)
- Other will cube a given number (as parameter)
- Parameters are passed as args (similar to args concept of Threading)
- Make sure that only one process prints to the standard output at a time 

In [95]:
import os
import multiprocessing

def square(num,lock):
    with lock:
        print(f'Square process id: {os.getpid()}')
        print(f'Square of {num} is {num ** 2}')

def cube(num,lock):
    with lock:
        print(f'Cube process id: {os.getpid()}')
        print(f'Cube of {num} is {num ** 3}')    
#---------------------------------------------

if __name__ == '__main__':
    print(f'Main Process , Process id: {os.getpid()}')

    lock = multiprocessing.Lock()
    p1 = multiprocessing.Process(target=square, args=(10,lock))
    p2 = multiprocessing.Process(target=cube, args=(10,lock))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print('End of Main Process')

Main Process , Process id: 1930
Square process id: 2532
Square of 10 is 100
Cube process id: 2535
Cube of 10 is 1000
End of Main Process


#### Sharing data in MultiProcessing through a Queue

In [98]:
from multiprocessing import Process, Queue

# Child process code
def func(queue):
    value = queue.get()
    print('Child got -', value)
    
    for i in range(1,value+1):
        queue.put(i ** 2)
    
#------------------------------
if __name__ == '__main__':
    q = Queue()
    p = Process(target=func, args=(q,))
    q.put(5)
    p.start()
    p.join()

    print('--------------')
    while not q.empty():
        print(q.get())

Child got - 5
--------------
1
4
9
16
25


#### Sharing data in MultiProcessing through a Pipe

In [99]:
from multiprocessing import Process, Pipe

# Child process code
def func(conn):
    value = conn.recv()
    print('Child got -', value)
    
    for i in range(1,value+1):
        conn.send(i ** 2)
#------------------------------

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=func, args=(child_conn,))
    p.start()
    
    num = 10
    parent_conn.send(num)
    
    for i in range(num):
        print(parent_conn.recv())
    
    p.join()
    parent_conn.close() # Pipe is closed, communication is no more possible
    print('Pipe Closed')

Child got - 10
1
4
9
16
25
36
49
64
81
100
Pipe Closed
